# Analyzing Video Metadata

### Part 1: Analyzing Tags
- tag vs number of videos: most videos, least videos
- tag vs duration: most duration, least duration, average duration

### Part 2: Analyzing Categories

#### Topic Modelling 
- Unsupervised Topic Clustering to figure out which tags belong together. 
- Assigning an appropriate category manually.
- Assigning Categories to each video based on it's tags list

#### Category analysis
- category vs number of videos: most videos, least videos
- category vs duration: most duration, least duration, average duration

---

## Imports

In [1]:
import numpy as np
import pandas as pd
import isodate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt

---

## Reading and Understanding the Data

In [2]:
df = pd.read_json('video_relevant_data.json', orient='index')

In [3]:
df.head(5)

,id,publishedAt,tags,categoryId,duration,viewCount,likeCount,commentCount,topicCategories
gOR9qZ3ZgwA,gOR9qZ3ZgwA,2020-07-15T13:00:14Z,"[Víctor Robles, Victor Robles WEB, victorroble...",27,PT19M48S,436665,16089,422,[https://en.wikipedia.org/wiki/Knowledge]
rC7X3nCwdZ8,rC7X3nCwdZ8,2019-05-14T22:00:04Z,"[votw, froglogic, squish, gui testing, gui aut...",28,PT3M57S,5067,14,3,[https://en.wikipedia.org/wiki/Knowledge]
WTLPmUHTPqo,WTLPmUHTPqo,2022-02-20T16:00:14Z,"[tech with tim, planet simulation in python, h...",27,PT1H2S,84771,3567,186,[https://en.wikipedia.org/wiki/Knowledge]
RjhiMjY1nI0,RjhiMjY1nI0,2021-06-18T12:00:10Z,"[python tutorial ita, tutorial python ita, tut...",27,PT9M28S,9726,250,64,[https://en.wikipedia.org/wiki/Knowledge]
YJyIykLyOmo,YJyIykLyOmo,2020-05-08T15:14:00Z,"[python bank management system project, python...",27,PT8M54S,2285,43,6,[https://en.wikipedia.org/wiki/Knowledge]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 458 entries, gOR9qZ3ZgwA to Y8Tko2YC5hA
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               458 non-null    object
 1   publishedAt      458 non-null    object
 2   tags             458 non-null    object
 3   categoryId       458 non-null    int64 
 4   duration         458 non-null    object
 5   viewCount        458 non-null    int64 
 6   likeCount        458 non-null    int64 
 7   commentCount     458 non-null    int64 
 8   topicCategories  458 non-null    object
dtypes: int64(4), object(5)
memory usage: 35.8+ KB


In [5]:
df.describe()

,categoryId,viewCount,likeCount,commentCount
count,458.000000,4.580000e+02,458.000000,458.000000
mean,24.707424,1.798409e+06,24443.908297,1317.021834
std,5.705154,7.613505e+06,70015.406159,5300.866314
min,1.000000,1.000000e+00,0.000000,0.000000
25%,26.000000,1.693025e+04,421.250000,37.000000
50%,27.000000,1.062635e+05,3614.500000,171.000000
75%,27.000000,8.252188e+05,20034.000000,847.500000
max,28.000000,1.282673e+08,778630.000000,84449.000000


In [6]:
df.shape

(458, 9)

### Data Preprocessing

In [7]:
# converting duration and publishedAt to datetime
df['duration'] = df['duration'].apply(lambda x: isodate.parse_duration(x))
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

In [8]:
# dropping id column as index already has that value
df.drop('id', axis=1, inplace=True)

In [9]:
df.head(2)

,publishedAt,tags,categoryId,duration,viewCount,likeCount,commentCount,topicCategories
gOR9qZ3ZgwA,2020-07-15 13:00:14+00:00,"[Víctor Robles, Victor Robles WEB, victorroble...",27,0 days 00:19:48,436665,16089,422,[https://en.wikipedia.org/wiki/Knowledge]
rC7X3nCwdZ8,2019-05-14 22:00:04+00:00,"[votw, froglogic, squish, gui testing, gui aut...",28,0 days 00:03:57,5067,14,3,[https://en.wikipedia.org/wiki/Knowledge]


In [10]:
# fetching complete list of unique tags from tags column
all_tags = list(set(",".join(df['tags'].apply(lambda x: ",".join(x)).to_list()).split(",")))

In [11]:
len(all_tags)

5695

In [12]:
print(all_tags[0:10])

['khatron ke khiladi 2020', 'what is pong', 'sensorpush', 'python tutorials for absolute beginners in hindi', 'python циклы', 'προγραμματιστμός', 'sagar', 'ilk işim', 'tutoriel python - les bases', 'what']


In [13]:
# creating seperate column for each unique tag for further analysis
for tag in all_tags:
    df[tag] = df['tags'].apply(lambda x: tag in x).map(int)

In [14]:
df.shape

(458, 5703)

In [15]:
df.head(1)

,publishedAt,tags,categoryId,duration,viewCount,likeCount,commentCount,topicCategories,khatron ke khiladi 2020,what is pong,...,for in range,começar no python,wild animals world,Socratica,python scripting for devops,python tamil book,python for beginners,9x19 mm,pandas tutorial in hindi,environment
gOR9qZ3ZgwA,2020-07-15 13:00:14+00:00,"[Víctor Robles, Victor Robles WEB, victorroble...",27,0 days 00:19:48,436665,16089,422,[https://en.wikipedia.org/wiki/Knowledge],0,0,...,0,0,0,0,0,0,0,0,0,0


---

### Analyzing tags

#### Video Count
- tag vs video count
- tag with most videos
- tag with least videos

In [16]:
tag_vs_vid_count_df = pd.DataFrame(df.iloc[:, 8:].sum().sort_values(ascending=False), columns=["total_vid_count"])

In [17]:
tag_vs_vid_count_df.head(2)

,total_vid_count
python,199
python tutorial,80


In [18]:
tag_vs_vid_count_df.tail(2)

,total_vid_count
بايثون ويب,1
khatron ke khiladi 2020,1


In [19]:
tag_vs_vid_count_df.to_csv('tags_vs_video_count.csv', index_label=['Tag'])

In [20]:
# tags with max videos
tag_vs_vid_count_df[tag_vs_vid_count_df['total_vid_count'] == tag_vs_vid_count_df['total_vid_count'].max()]

,total_vid_count
python,199


In [21]:
# tags with min videos
tag_vs_vid_count_df[tag_vs_vid_count_df['total_vid_count'] == tag_vs_vid_count_df['total_vid_count'].min()]

,total_vid_count
python 2019,1
anaconda movie,1
tree structure,1
academy,1
crocodile python vevo official music video,1
...,...
pubg lite funny gameplay,1
Agriculture,1
data structures,1
بايثون ويب,1


---

#### Duration
- tag vs duration
- tag with most duration
- tag with least duration
- average duration for each tag

In [22]:
tag_vs_duration = {}

for column in df.iloc[:, 8:]:
    average_duration = df[df[column] == 1]['duration'].sum()/len(df[df[column] == 1])
    total_duration_for_tag = df[df[column] == 1]['duration'].sum()
    tag_vs_duration[column] = [average_duration, total_duration_for_tag]

tag_vs_duration_df = pd.DataFrame.from_dict(tag_vs_duration, orient='index', columns=['avg_duration', 'total_duration'])

In [23]:
tag_vs_duration_df.head(1)

,avg_duration,total_duration
khatron ke khiladi 2020,0 days 00:02:45,0 days 00:02:45


In [24]:
tag_vs_count_duration_df = tag_vs_vid_count_df.join(tag_vs_duration_df)

In [25]:
tag_vs_count_duration_df.head(2)

,total_vid_count,avg_duration,total_duration
python,199,0 days 01:06:50.080402010,9 days 05:40:06
python tutorial,80,0 days 02:01:50.025000,6 days 18:26:42


In [26]:
# max total duration
tag_vs_duration_df[tag_vs_duration_df['total_duration'] == tag_vs_duration_df['total_duration'].max()]

,avg_duration,total_duration
python,0 days 01:06:50.080402010,9 days 05:40:06


In [27]:
# min total duration
tag_vs_duration_df[tag_vs_duration_df['total_duration'] == tag_vs_duration_df['total_duration'].min()]

,avg_duration,total_duration
leaf media,0 days 00:00:10,0 days 00:00:10
Monty Python And The Holy Grail,0 days 00:00:10,0 days 00:00:10


In [28]:
# max average duration
tag_vs_duration_df[tag_vs_duration_df['avg_duration'] == tag_vs_duration_df['avg_duration'].max()]

,avg_duration,total_duration
dr. chuck,0 days 13:40:10,0 days 13:40:10
py4e,0 days 13:40:10,0 days 13:40:10
university of michigan,0 days 13:40:10,0 days 13:40:10
u of m,0 days 13:40:10,0 days 13:40:10
python tutorial 2019,0 days 13:40:10,0 days 13:40:10
charles severance,0 days 13:40:10,0 days 13:40:10
dr chuck,0 days 13:40:10,0 days 13:40:10


In [29]:
# min average duration
tag_vs_duration_df[tag_vs_duration_df['avg_duration'] == tag_vs_duration_df['avg_duration'].min()]

,avg_duration,total_duration
leaf media,0 days 00:00:10,0 days 00:00:10
Monty Python And The Holy Grail,0 days 00:00:10,0 days 00:00:10


#### Storing Tag vs Count & Duration Information in csv file

In [30]:
tag_vs_count_duration_df.to_csv('tag_analysis.csv', index_label=['Tag'])

---

### Category Analysis

#### Topic Modelling
- To group tags into categories we need to perform topic modelling i.e. unsupervised clustering of tags
- Then we need to manually assign relevant categories to topics

In [31]:
# converting list to string for topic modelling
df["tags_str"] = df['tags'].apply(lambda x: " ".join(x))

In [32]:
vect = TfidfVectorizer(stop_words='english')
X = vect.fit_transform(df["tags_str"])

pd.DataFrame(X.toarray(), columns=vect.get_feature_names())[0:10]

,10,100,1000,1000iqgameplays,100本ノック,1080p,11,12,15,16,...,코딩독학,코딩테스트,크롤링,테스트,파이썬,파이썬강의,파이썬강좌,파이썬공부,파이썬독학,프로그래밍
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222185,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# choosing 5 as the number of topics (clusters)
N_TOPICS = 8
nmf = NMF(n_components=N_TOPICS, init='nndsvd')
W = nmf.fit_transform(X)  # Document-topic matrix
H = nmf.components_       # Topic-term matrix

In [34]:
NUM_TOP_WORDS_TO_SHOW = 10
words = np.array(vect.get_feature_names())
topic_words = pd.DataFrame(np.zeros((N_TOPICS, NUM_TOP_WORDS_TO_SHOW)), 
                           index=[f'Topic {i + 1}' for i in range(N_TOPICS)],
                           columns=[f'Word {i + 1}' for i in range(NUM_TOP_WORDS_TO_SHOW)]
                          ).astype(str)

for i in range(N_TOPICS):
    ix = H[i].argsort()[::-1][:NUM_TOP_WORDS_TO_SHOW]
    topic_words.iloc[i] = words[ix]

topic_words

,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10
Topic 1,python,programming,learn,tutorial,beginners,course,language,development,coding,software
Topic 2,python,curso,tutorial,windows,programacion,install,aprender,corso,em,en
Topic 3,telusko,navin,reddy,java,tutorial,set,python,robotics,google,ai
Topic 4,movie,snake,adventure,free,chinese,king,action,movies,horror,youku
Topic 5,data,learning,science,machine,pandas,analysis,intelligence,artificial,python,numpy
Topic 6,hindi,python,tutorial,beginners,learn,course,complete,video,class,pandas
Topic 7,для,уроки,питон,нуля,начинающих,программирование,python,пайтон,как,pygame
Topic 8,animals,snake,animal,national,wild,wildlife,geographic,nat,geo,vs


In [35]:
# assigning topics
topic_mapping = {
    'Topic 1': 'python tutorials for beginners',
    'Topic 2': 'python tutorials in spanish',
    'Topic 3': 'general purpose programming',
    'Topic 4': 'movies',
    'Topic 5': 'machine learning and ai',
    'Topic 6': 'python tutorials in hindi',
    'Topic 7': 'python tutorials in russian',
    'Topic 8': 'wildlife'
}

In [36]:
W = pd.DataFrame(W, columns=[f'Topic {i + 1}' for i in range(N_TOPICS)])
W['max_topic'] = W.apply(lambda x: topic_mapping.get(x.idxmax()), axis=1)
W[pd.notnull(W['max_topic'])].head(2)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,max_topic
0,0.00000,0.298835,0.003911,0.000000,0.0,0.000000,0.0,0.0,python tutorials in spanish
1,0.02653,0.041082,0.017935,0.009143,0.0,0.003675,0.0,0.0,python tutorials in spanish


In [37]:
df['category'] = W['max_topic'].to_list()

In [38]:
df.head(2)

,publishedAt,tags,categoryId,duration,viewCount,likeCount,commentCount,topicCategories,khatron ke khiladi 2020,what is pong,...,wild animals world,Socratica,python scripting for devops,python tamil book,python for beginners,9x19 mm,pandas tutorial in hindi,environment,tags_str,category
gOR9qZ3ZgwA,2020-07-15 13:00:14+00:00,"[Víctor Robles, Victor Robles WEB, victorroble...",27,0 days 00:19:48,436665,16089,422,[https://en.wikipedia.org/wiki/Knowledge],0,0,...,0,0,0,0,0,0,0,0,Víctor Robles Victor Robles WEB victorrobleswe...,python tutorials in spanish
rC7X3nCwdZ8,2019-05-14 22:00:04+00:00,"[votw, froglogic, squish, gui testing, gui aut...",28,0 days 00:03:57,5067,14,3,[https://en.wikipedia.org/wiki/Knowledge],0,0,...,0,0,0,0,0,0,0,0,votw froglogic squish gui testing gui automati...,python tutorials in spanish


#### Analyzing Categories
- category vs total videos
- category vs duration

In [39]:
category_df = pd.DataFrame(df['category'].value_counts())
category_df

,category
python tutorials in spanish,197
python tutorials for beginners,73
wildlife,62
machine learning and ai,40
python tutorials in russian,32
python tutorials in hindi,22
general purpose programming,18
movies,14


In [40]:
# Most Popular Category
category_df[category_df.category == category_df.category.max()]

,category
python tutorials in spanish,197


In [41]:
# Least Popular Category
category_df[category_df.category == category_df.category.min()]

,category
movies,14


In [42]:
total_duration_cat_df = pd.DataFrame(df.groupby('category')['duration'].sum())
total_duration_cat_df

,duration
category,
general purpose programming,1 days 02:24:26
machine learning and ai,2 days 10:14:18
movies,0 days 08:43:25
python tutorials for beginners,4 days 23:13:41
python tutorials in hindi,2 days 04:22:11
python tutorials in russian,0 days 23:51:31
python tutorials in spanish,6 days 07:49:27
wildlife,0 days 09:16:00


In [43]:
avg_duration_cat_df = pd.DataFrame(df.groupby('category')['duration'].sum().sort_values(ascending=False)/df['category'].value_counts(), columns=['avg_duration'])
avg_duration_cat_df

,avg_duration
general purpose programming,0 days 01:28:01.444444444
machine learning and ai,0 days 01:27:21.450000
movies,0 days 00:37:23.214285714
python tutorials for beginners,0 days 01:37:59.739726027
python tutorials in hindi,0 days 02:22:49.590909090
python tutorials in russian,0 days 00:44:44.093750
python tutorials in spanish,0 days 00:46:14.451776649
wildlife,0 days 00:08:58.064516129


In [44]:
total_duration_cat_df.join(avg_duration_cat_df).sort_values(by='avg_duration', ascending=False)

,duration,avg_duration
category,,
python tutorials in hindi,2 days 04:22:11,0 days 02:22:49.590909090
python tutorials for beginners,4 days 23:13:41,0 days 01:37:59.739726027
general purpose programming,1 days 02:24:26,0 days 01:28:01.444444444
machine learning and ai,2 days 10:14:18,0 days 01:27:21.450000
python tutorials in spanish,6 days 07:49:27,0 days 00:46:14.451776649
python tutorials in russian,0 days 23:51:31,0 days 00:44:44.093750
movies,0 days 08:43:25,0 days 00:37:23.214285714
wildlife,0 days 09:16:00,0 days 00:08:58.064516129


In [45]:
total_duration_cat_df.to_csv("category_analysis.csv")

---

In [ ]:
df['category'].value